# Data preparation and EDA



In [ ]:
!pip install emoji
!python -m spacy download ru_core_news_md
!pip install pymystem3
!pip install -q transformers datasets librosa

     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 41.9/41.9 MB 20.9 MB/s eta 0:00:00
‚úî Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
‚ö† Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
from torch.utils.data import Dataset
import torch
import string
import re
import emoji
import ru_core_news_md
from string import whitespace

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from collections import defaultdict

from pymystem3 import Mystem
from datasets import load_dataset

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
df = pd.read_csv('valid.csv')

In [ ]:
df.dtypes

text        object
anger        int64
disgust      int64
fear         int64
joy          int64
sadness      int64
surprise     int64
neutral      int64
dtype: object

In [ ]:
df.head()

text  anger  disgust  fear  \
0  –≠—Ç–æ –≤ –ù–æ–≤–æ–º –û—Ä–ª–µ–∞–Ω–µ?? –Ø –¥–µ–π—Å—Ç–≤–∏—Ç–µ–ª—å–Ω–æ —á—É–≤—Å—Ç–≤—É—é...      0        0     0   
1  –¢—ã –∑–Ω–∞–µ—à—å –æ—Ç–≤–µ—Ç, —á—É–≤–∞–∫, —Ç—ã –∑–∞–ø—Ä–æ–≥—Ä–∞–º–º–∏—Ä–æ–≤–∞–Ω –ø–µ...      0        0     0   
2           –ú–Ω–µ –Ω–∏–∫–æ–≥–¥–∞ –≤ –∂–∏–∑–Ω–∏ –Ω–µ –±—ã–ª–æ —Ç–∞–∫ –≥—Ä—É—Å—Ç–Ω–æ!      0        0     0   
3  –≠–∫–æ–Ω–æ–º–∏–∫–∞ –Ω–∞—Ö–æ–¥–∏—Ç—Å—è –ø–æ–¥ –∂–µ—Å—Ç–∫–∏–º –∫–æ–Ω—Ç—Ä–æ–ª–µ–º –∏ —Å—É...      0        0     0   
4  –û–Ω –º–æ–≥ –±—ã –ª–µ–≥–∫–æ –≤–∑—è—Ç—å –Ω–∞—Å—Ç–æ—è—â—É—é –∫–∞–º–µ—Ä—É –∏–∑ –∑–∞–∫–æ...      0        0     0   

   joy  sadness  surprise  neutral  
0    0        0         0        1  
1    1        0         0        1  
2    0        1         0        0  
3    1        0         0        1  
4    1        0         0        0

In [ ]:
df.shape

(5426, 8)

In [ ]:
df.nunique()

text        5416
anger          2
disgust        2
fear           2
joy            2
sadness        2
surprise       2
neutral        2
dtype: int64

In [ ]:
df['text'].value_counts()

text
–ú–Ω–µ —ç—Ç–æ –Ω—Ä–∞–≤–∏—Ç—Å—è!                                                                                                                                                      3
–°–ø–∞—Å–∏–±–æ!                                                                                                                                                               3
–°–ü–ê–°–ò–ë–û [–ò–ú–Ø]                                                                                                                                                          2
–ë–æ–ª—å—à–æ–µ —Å–ø–∞—Å–∏–±–æ.                                                                                                                                                       2
–Ø –≤–∑–≤–æ–ª–Ω–æ–≤–∞–Ω!                                                                                                                                                          2
                                                                                                                                                                      ..
–Ø —Ç–∞–∫ —Ä–∞–¥, —á—Ç–æ –≤—ã –¥–æ–±–∏–ª–∏—Å—å —Å–ø—Ä–∞–≤–µ–¥–ª–∏–≤–æ—Å—Ç–∏ –∏ –∑–∞–∫—Ä—ã—Ç–∏—è. –í—ã –º–æ–∂–µ—Ç–µ –Ω–∞—á–∞—Ç—å –ø—Ä–∞–≤–∏–ª—å–Ω–æ –ª–µ—á–∏—Ç—å—Å—è —É–∂–µ —Å–µ–π—á–∞—Å. –Ø –Ω–µ —É–≤–µ—Ä–µ–Ω, —á—Ç–æ –∫–æ–≥–¥–∞-–Ω–∏–±—É–¥—å –ø–æ–ª—É—á—É —Ç–æ –∂–µ —Å–∞–º–æ–µ, –∫ —Å–æ–∂–∞–ª–µ–Ω–∏—é    1
[–ò–ú–Ø] –ø—Ä–æ—Å—Ç–æ –ø–µ—Ä–µ—Å—Ç–∞–ª –ø–æ–ª—É—á–∞—Ç—å –º—è—á –≤ 4-–π —á–µ—Ç–≤–µ—Ä—Ç–∏ –∏ –≤—Å—Ç–∞–ª –≤ —É–≥–ª—É                                                                                                       1
–ö —Å–æ–∂–∞–ª–µ–Ω–∏—é, —ç—Ç–æ—Ç —Å–∞–± –ø–æ—Å—Ç–µ–ø–µ–Ω–Ω–æ –∑–∞—Å–µ–ª—è–µ—Ç—Å—è —Ä–µ–≤–∏–∑–∏–æ–Ω–∏—Å—Ç–∞–º–∏ –ø—Ä–∏–∫–≤–µ–ª–æ–≤ –∏ —Å—Ç–∞—Ä—ã–º–∏ –¥–æ–±—Ä—ã–º–∏ –ø–∞—Ä–Ω—è–º–∏.                                                                        1
–Ω–∞–¥–µ—é—Å—å –Ω–∞ —ç—Ç–æ! Gfinity3 –≤ Copperbox –±—ã–ª –Ω–µ–≤–µ—Ä–æ—è—Ç–Ω—ã–º —Å–æ–±—ã—Ç–∏–µ–º –∏ –∞—Ç–º–æ—Å—Ñ–µ—Ä–æ–π.                                                                                            1
::–Ω–æ –º–Ω–µ –Ω—Ä–∞–≤–∏—Ç—Å—è –¥–µ—Ç—Å–∫–∞—è —á–µ–ª–∫–∞:: /tiny voice                                                                                                                          1
Name: count, Length: 5416, dtype: int64

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

(5417, 8)

In [ ]:
tatget_columns = ['anger',	'disgust',	'fear',	'joy',	'sadness',	'surprise',	'neutral']

# Functions for feature extruction

In [ ]:
def extract_hashtags(text: str) -> str:
  '''
  Get hashtags from text
  '''
  hashtags = re.findall(r'#\w+', text)
  return hashtags

In [ ]:
def extract_mentions(text: str) -> str:
  '''
  Get mentioned person from text
  '''
  mentions = re.findall(r'@\w+', text)
  return mentions

In [ ]:
def get_lenght(text: str) -> int:
  '''
  Get length of text
  '''
  return len(text)

In [ ]:
def count_exclamation(text: str) -> int:
  '''
  Count exclamation marks
  '''
  return text.count('!')

In [ ]:
def count_question(text: str) -> int:
  '''
  Count question marks
  '''
  return text.count('?')

In [ ]:
def count_positive_brackets(text: str) -> int:
  '''
  Calculate "positive" brackets
  '''
  return text.count(')')

In [ ]:
def count_negative_brackets(text: str) -> int:
  '''
  Calculate "negative" brackets
  '''
  return text.count('(')

In [ ]:
def get_emoji(text: str) -> str:
  '''
  Extract list of emojis
  '''
  res = []
  emojis = [emoji.emojize(word) for word in text.split()]
  for em in emojis:
    if emoji.is_emoji(em):
      res.append(em)
  return ' '.join(res)

In [ ]:
def get_num_emoji(emojies_list: list) -> int:
  '''
  Get number of emojies
  '''
  return len(emojies_list)

In [ ]:
def demojize(emojis_list: list) -> str:
  '''
  Convert emojis to text
  '''
  return ' '.join([emoji.demojize(em) for em in emojis_list])

In [ ]:
def get_url(text: str) -> str:
  '''
  Get urls from text.
  '''
  url = re.findall(r'https?://\S+', text)
  return url

In [ ]:
def get_word_count(text: str) -> int:
  '''
  Count number of words.
  '''
  return len(text.split())

In [ ]:
def get_avg_word_length(text: str) -> int:
  '''
  Count avarage lenght of words.
  '''
  words = text.split()
  word_count = len(words)
  avg_word_lengh = sum(len(word) for word in words) / word_count
  return avg_word_lengh

In [ ]:
def get_capitalized_words(text: str) -> str:
  '''
  Extract capitalazed words
  '''
  capitalazed_words = [word for word in text.split() if word.upper()]
  return ' '.join(capitalazed_words)

In [ ]:
def get_names(text: str) -> str:
  '''
  Extract named entities.
  '''
  nlp = ru_core_news_md.load()
  document = nlp(text)
  names = document.ents
  return ' '.join(names)

In [ ]:
#TODO Build pipeline
df['hashtags'] = df.text.map(extract_hashtags)
df['mentions'] = df.text.map(extract_mentions)
df['text_length'] = df.text.map(get_lenght)
df['num_exclamation'] = df.text.map(count_exclamation)
df['num_question'] = df.text.map(count_question)
df['num_positive_brackets'] = df.text.map(count_positive_brackets)
df['num_negative_brackets'] = df.text.map(count_negative_brackets)
df['emoji_list'] = df.text.map(get_emoji)
df['num_emojis'] = df.emoji_list.map(get_num_emoji)
df['emogies_as_word'] = df.emoji_list.map(demojize)
df['url'] = df.text.map(get_url)
df['num_words'] = df.text.map(get_word_count)
df['avg_word'] = df.text.map(get_avg_word_length)

# Clean text

In [ ]:
def handle_punctuation(inp_str: str) -> str:
  inp_str = str(inp_str)
  for punct in string.punctuation:
    inp_str = inp_str.replace(punct, ' ')
    inp_str = inp_str.replace('‚Äî', ' ')
    inp_str = inp_str.strip().lower()
  return inp_str


In [ ]:
cyrillic_letters = u"–∞–±–≤–≥–¥–µ—ë–∂–∑–∏–π–∫–ª–º–Ω–æ–ø—Ä—Å—Ç—É—Ñ—Ö—Ü—á—à—â—ä—ã—å—ç—é—è"

def remove_trash(text):
    allowed_chars = cyrillic_letters + whitespace
    return "".join([c for c in text if c in allowed_chars])

In [ ]:
# def lemma_text(text: str):
#   mystem = Mystem()
#   lemmas = mystem.lemmatize(text)
#   res = ''.join(lemmas)
#   return res

In [ ]:
def lemma_text(texts: str):
  mystem = Mystem()
  lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
  txtpart = lol(texts, 1000)
  res = []
  for txtp in txtpart:
    alltexts = ' '.join([txt + ' br ' for txt in txtp])

    words = mystem.lemmatize(alltexts)
    doc = []
    for txt in words:
        if txt != '\n' and txt.strip() != '':
            if txt == 'br':
                res.append(doc)
                doc = []
            else:
                doc.append(txt)
  return res

In [ ]:
def del_stopwords(text: str):
  tokens = word_tokenize(text)
  stop_words = set(stopwords.words('russian'))
  filtered_tokens = [word for word in tokens if word not in stop_words]
  return filtered_tokens

In [ ]:
df.text = df.text.map(handle_punctuation)
df.text = df.text.map(remove_trash)
df.text = df.text.map(del_stopwords)

In [ ]:
df.text = df.text.str.join(' ')

In [ ]:
df.text = lemma_text(df.text)

In [ ]:
df

text  anger  disgust  fear  \
0     [—ç—Ç–æ, –Ω–æ–≤—ã–π, –æ—Ä–ª–µ–∞–Ω, –¥–µ–π—Å—Ç–≤–∏—Ç–µ–ª—å–Ω–æ, —á—É–≤—Å—Ç–≤–æ–≤–∞—Ç...      0        0     0   
1     [–∑–Ω–∞—Ç—å, –æ—Ç–≤–µ—Ç, —á—É–≤–∞–∫, –∑–∞–ø—Ä–æ–≥—Ä–∞–º–º–∏—Ä–æ–≤–∞—Ç—å, –ø–µ—Ä–µ—Ö...      0        0     0   
2                                      [–∂–∏–∑–Ω—å, –≥—Ä—É—Å—Ç–Ω–æ]      0        0     0   
3     [—ç–∫–æ–Ω–æ–º–∏–∫–∞, –Ω–∞—Ö–æ–¥–∏—Ç—å—Å—è, –∂–µ—Å—Ç–∫–∏–π, –∫–æ–Ω—Ç—Ä–æ–ª—å, —Å—É–±...      0        0     0   
4     [–º–æ—á—å, –ª–µ–≥–∫–æ, –≤–∑—è—Ç—å, –Ω–∞—Å—Ç–æ—è—â–∏–π, –∫–∞–º–µ—Ä–∞, –∑–∞–∫–æ–Ω–Ω...      0        0     0   
...                                                 ...    ...      ...   ...   
5421  [—ç—Ç–æ, –¥–æ–≤–æ–ª—å–Ω–æ, –æ–ø–∞—Å–Ω—ã–π, –≥–æ—Å—É–¥–∞—Ä—Å—Ç–≤–æ, —Ä–µ—à–∞—Ç—å, ...      0        0     1   
5422  [—Å–µ–≥–æ–¥–Ω—è, —É—Ç—Ä–æ, –ø–æ–¥–∞–≤–∞—Ç—å, —Ä–∞–∑–≤–æ–¥, –Ω–∞–¥–µ—è—Ç—å—Å—è, —É...      0        0     0   
5423  [–ø–æ—Å–ª–µ–¥–Ω–∏–π, —ç—Ç–æ, —Å–ª—É—á–∞—Ç—å—Å—è, –ø—Ä–æ—Å—Ç–æ, —Å–∫–∞–∑–∞—Ç—å, –∑...      1        0     0   
5424  [—Ç–µ—Ä–ø–µ—Ç—å, –º–æ—á—å, –≤—ã—Å–æ–∫–æ–º–µ—Ä–Ω—ã–π, –ø—Ä–∏–¥—É—Ä–æ–∫, —Ñ–æ—Ä–¥, ...      1        0     0   
5425                        [–Ω—Ä–∞–≤–∏—Ç—å—Å—è, –¥–µ—Ç—Å–∫–∏–π, —á–µ–ª–∫–∞]      0        0     0   

      joy  sadness  surprise  neutral hashtags mentions  ...  num_exclamation  \
0       0        0         0        1       []       []  ...                0   
1       1        0         0        1       []       []  ...                1   
2       0        1         0        0       []       []  ...                1   
3       1        0         0        1       []       []  ...                0   
4       1        0         0        0       []       []  ...                0   
...   ...      ...       ...      ...      ...      ...  ...              ...   
5421    0        0         0        0       []       []  ...                0   
5422    1        0         0        0       []       []  ...                0   
5423    0        0         0        0       []       []  ...                0   
5424    0        0         0        0       []       []  ...                0   
5425    1        0         0        0       []       []  ...                0   

      num_question  num_positive_brackets  num_negative_brackets  emoji_list  \
0                2                      0                      0               
1                0                      0                      0               
2                0                      0                      0               
3                0                      0                      0               
4                0                      0                      0               
...            ...                    ...                    ...         ...   
5421             0                      0                      0               
5422             0                      0                      0               
5423             0                      0                      0               
5424             0                      0                      0               
5425             0                      0                      0               

     num_emojis  emogies_as_word url num_words  avg_word  
0             0                   []        11  5.363636  
1             0                   []        16  5.812500  
2             0                   []         8  4.125000  
3             0                   []        20  6.250000  
4             0                   []        18  5.611111  
...         ...              ...  ..       ...       ...  
5421          0                   []        13  7.615385  
5422          0                   []        15  4.733333  
5423          0                   []        13  4.846154  
5424          0                   []        17  4.588235  
5425          0                   []         7  5.571429  

[5417 rows x 21 columns]

In [ ]:
#df.to_csv('valid_lemma_df.csv')

In [ ]:
# from google.colab import files
# files.download('valid_lemma_df.csv')

In [ ]:
df

text  anger  disgust  fear  \
0     [—ç—Ç–æ, –Ω–æ–≤—ã–π, –æ—Ä–ª–µ–∞–Ω, –¥–µ–π—Å—Ç–≤–∏—Ç–µ–ª—å–Ω–æ, —á—É–≤—Å—Ç–≤–æ–≤–∞—Ç...      0        0     0   
1     [–∑–Ω–∞—Ç—å, –æ—Ç–≤–µ—Ç, —á—É–≤–∞–∫, –∑–∞–ø—Ä–æ–≥—Ä–∞–º–º–∏—Ä–æ–≤–∞—Ç—å, –ø–µ—Ä–µ—Ö...      0        0     0   
2                                      [–∂–∏–∑–Ω—å, –≥—Ä—É—Å—Ç–Ω–æ]      0        0     0   
3     [—ç–∫–æ–Ω–æ–º–∏–∫–∞, –Ω–∞—Ö–æ–¥–∏—Ç—å—Å—è, –∂–µ—Å—Ç–∫–∏–π, –∫–æ–Ω—Ç—Ä–æ–ª—å, —Å—É–±...      0        0     0   
4     [–º–æ—á—å, –ª–µ–≥–∫–æ, –≤–∑—è—Ç—å, –Ω–∞—Å—Ç–æ—è—â–∏–π, –∫–∞–º–µ—Ä–∞, –∑–∞–∫–æ–Ω–Ω...      0        0     0   
...                                                 ...    ...      ...   ...   
5421  [—ç—Ç–æ, –¥–æ–≤–æ–ª—å–Ω–æ, –æ–ø–∞—Å–Ω—ã–π, –≥–æ—Å—É–¥–∞—Ä—Å—Ç–≤–æ, —Ä–µ—à–∞—Ç—å, ...      0        0     1   
5422  [—Å–µ–≥–æ–¥–Ω—è, —É—Ç—Ä–æ, –ø–æ–¥–∞–≤–∞—Ç—å, —Ä–∞–∑–≤–æ–¥, –Ω–∞–¥–µ—è—Ç—å—Å—è, —É...      0        0     0   
5423  [–ø–æ—Å–ª–µ–¥–Ω–∏–π, —ç—Ç–æ, —Å–ª—É—á–∞—Ç—å—Å—è, –ø—Ä–æ—Å—Ç–æ, —Å–∫–∞–∑–∞—Ç—å, –∑...      1        0     0   
5424  [—Ç–µ—Ä–ø–µ—Ç—å, –º–æ—á—å, –≤—ã—Å–æ–∫–æ–º–µ—Ä–Ω—ã–π, –ø—Ä–∏–¥—É—Ä–æ–∫, —Ñ–æ—Ä–¥, ...      1        0     0   
5425                        [–Ω—Ä–∞–≤–∏—Ç—å—Å—è, –¥–µ—Ç—Å–∫–∏–π, —á–µ–ª–∫–∞]      0        0     0   

      joy  sadness  surprise  neutral hashtags mentions  ...  num_exclamation  \
0       0        0         0        1       []       []  ...                0   
1       1        0         0        1       []       []  ...                1   
2       0        1         0        0       []       []  ...                1   
3       1        0         0        1       []       []  ...                0   
4       1        0         0        0       []       []  ...                0   
...   ...      ...       ...      ...      ...      ...  ...              ...   
5421    0        0         0        0       []       []  ...                0   
5422    1        0         0        0       []       []  ...                0   
5423    0        0         0        0       []       []  ...                0   
5424    0        0         0        0       []       []  ...                0   
5425    1        0         0        0       []       []  ...                0   

      num_question  num_positive_brackets  num_negative_brackets  emoji_list  \
0                2                      0                      0               
1                0                      0                      0               
2                0                      0                      0               
3                0                      0                      0               
4                0                      0                      0               
...            ...                    ...                    ...         ...   
5421             0                      0                      0               
5422             0                      0                      0               
5423             0                      0                      0               
5424             0                      0                      0               
5425             0                      0                      0               

     num_emojis  emogies_as_word url num_words  avg_word  
0             0                   []        11  5.363636  
1             0                   []        16  5.812500  
2             0                   []         8  4.125000  
3             0                   []        20  6.250000  
4             0                   []        18  5.611111  
...         ...              ...  ..       ...       ...  
5421          0                   []        13  7.615385  
5422          0                   []        15  4.733333  
5423          0                   []        13  4.846154  
5424          0                   []        17  4.588235  
5425          0                   []         7  5.571429  

[5417 rows x 21 columns]

In [ ]:
df

text  anger  disgust  fear  \
0     [—ç—Ç–æ, –Ω–æ–≤—ã–π, –æ—Ä–ª–µ–∞–Ω, –¥–µ–π—Å—Ç–≤–∏—Ç–µ–ª—å–Ω–æ, —á—É–≤—Å—Ç–≤–æ–≤–∞—Ç...      0        0     0   
1     [–∑–Ω–∞—Ç—å, –æ—Ç–≤–µ—Ç, —á—É–≤–∞–∫, –∑–∞–ø—Ä–æ–≥—Ä–∞–º–º–∏—Ä–æ–≤–∞—Ç—å, –ø–µ—Ä–µ—Ö...      0        0     0   
2                                      [–∂–∏–∑–Ω—å, –≥—Ä—É—Å—Ç–Ω–æ]      0        0     0   
3     [—ç–∫–æ–Ω–æ–º–∏–∫–∞, –Ω–∞—Ö–æ–¥–∏—Ç—å—Å—è, –∂–µ—Å—Ç–∫–∏–π, –∫–æ–Ω—Ç—Ä–æ–ª—å, —Å—É–±...      0        0     0   
4     [–º–æ—á—å, –ª–µ–≥–∫–æ, –≤–∑—è—Ç—å, –Ω–∞—Å—Ç–æ—è—â–∏–π, –∫–∞–º–µ—Ä–∞, –∑–∞–∫–æ–Ω–Ω...      0        0     0   
...                                                 ...    ...      ...   ...   
5421  [—ç—Ç–æ, –¥–æ–≤–æ–ª—å–Ω–æ, –æ–ø–∞—Å–Ω—ã–π, –≥–æ—Å—É–¥–∞—Ä—Å—Ç–≤–æ, —Ä–µ—à–∞—Ç—å, ...      0        0     1   
5422  [—Å–µ–≥–æ–¥–Ω—è, —É—Ç—Ä–æ, –ø–æ–¥–∞–≤–∞—Ç—å, —Ä–∞–∑–≤–æ–¥, –Ω–∞–¥–µ—è—Ç—å—Å—è, —É...      0        0     0   
5423  [–ø–æ—Å–ª–µ–¥–Ω–∏–π, —ç—Ç–æ, —Å–ª—É—á–∞—Ç—å—Å—è, –ø—Ä–æ—Å—Ç–æ, —Å–∫–∞–∑–∞—Ç—å, –∑...      1        0     0   
5424  [—Ç–µ—Ä–ø–µ—Ç—å, –º–æ—á—å, –≤—ã—Å–æ–∫–æ–º–µ—Ä–Ω—ã–π, –ø—Ä–∏–¥—É—Ä–æ–∫, —Ñ–æ—Ä–¥, ...      1        0     0   
5425                        [–Ω—Ä–∞–≤–∏—Ç—å—Å—è, –¥–µ—Ç—Å–∫–∏–π, —á–µ–ª–∫–∞]      0        0     0   

      joy  sadness  surprise  neutral hashtags mentions  ...  num_exclamation  \
0       0        0         0        1       []       []  ...                0   
1       1        0         0        1       []       []  ...                1   
2       0        1         0        0       []       []  ...                1   
3       1        0         0        1       []       []  ...                0   
4       1        0         0        0       []       []  ...                0   
...   ...      ...       ...      ...      ...      ...  ...              ...   
5421    0        0         0        0       []       []  ...                0   
5422    1        0         0        0       []       []  ...                0   
5423    0        0         0        0       []       []  ...                0   
5424    0        0         0        0       []       []  ...                0   
5425    1        0         0        0       []       []  ...                0   

      num_question  num_positive_brackets  num_negative_brackets  emoji_list  \
0                2                      0                      0               
1                0                      0                      0               
2                0                      0                      0               
3                0                      0                      0               
4                0                      0                      0               
...            ...                    ...                    ...         ...   
5421             0                      0                      0               
5422             0                      0                      0               
5423             0                      0                      0               
5424             0                      0                      0               
5425             0                      0                      0               

     num_emojis  emogies_as_word url num_words  avg_word  
0             0                   []        11  5.363636  
1             0                   []        16  5.812500  
2             0                   []         8  4.125000  
3             0                   []        20  6.250000  
4             0                   []        18  5.611111  
...         ...              ...  ..       ...       ...  
5421          0                   []        13  7.615385  
5422          0                   []        15  4.733333  
5423          0                   []        13  4.846154  
5424          0                   []        17  4.588235  
5425          0                   []         7  5.571429  

[5417 rows x 21 columns]

In [ ]:
df.head(30)

text  anger  disgust  fear  \
0   [—ç—Ç–æ, –Ω–æ–≤—ã–π, –æ—Ä–ª–µ–∞–Ω, –¥–µ–π—Å—Ç–≤–∏—Ç–µ–ª—å–Ω–æ, —á—É–≤—Å—Ç–≤–æ–≤–∞—Ç...      0        0     0   
1   [–∑–Ω–∞—Ç—å, –æ—Ç–≤–µ—Ç, —á—É–≤–∞–∫, –∑–∞–ø—Ä–æ–≥—Ä–∞–º–º–∏—Ä–æ–≤–∞—Ç—å, –ø–µ—Ä–µ—Ö...      0        0     0   
2                                    [–∂–∏–∑–Ω—å, –≥—Ä—É—Å—Ç–Ω–æ]      0        0     0   
3   [—ç–∫–æ–Ω–æ–º–∏–∫–∞, –Ω–∞—Ö–æ–¥–∏—Ç—å—Å—è, –∂–µ—Å—Ç–∫–∏–π, –∫–æ–Ω—Ç—Ä–æ–ª—å, —Å—É–±...      0        0     0   
4   [–º–æ—á—å, –ª–µ–≥–∫–æ, –≤–∑—è—Ç—å, –Ω–∞—Å—Ç–æ—è—â–∏–π, –∫–∞–º–µ—Ä–∞, –∑–∞–∫–æ–Ω–Ω...      0        0     0   
5   [—Å–ø–∞—Å–∏–±–æ, –¥–æ–≤–µ—Ä–∏–µ, —Å—Ç–∞—Ç–∏—Å—Ç–∏–∫–∞, –º–æ—á—å, –¥–æ—Å—Ç–∏–≥–∞—Ç—å...      0        0     0   
6   [–≤–∞—É, –º–∞–º–∞, –¥—Ä—É–≥–æ–π, —á–µ–ª–æ–≤–µ–∫, –Ω–∞–∑—ã–≤–∞—Ç—å, –º–æ–π, –µ—Ä...      1        0     0   
7                                               [–æ–Ω–æ]      0        0     0   
8   [–∫—Ä–∞–π–Ω–∏–π, –º–µ—Ä–∞, –∏–º—è, –≤—Ä–µ–º—è, –∑–∞–≤–æ–µ–≤—ã–≤–∞—Ç—å, –¥–æ–≤–µ—Ä–∏–µ]      0        0     0   
9      [—Ö–æ—Ä–æ—à–∏–π, —Ö–æ—Ç–µ—Ç—å, –º–∏—Ä, –æ—Ç–ø—Ä—ã—Å–∫, —Ç—Ä—ç—à, –ª–∏–±–µ—Ä–∞–ª]      1        0     0   
10  [—Å–∫–∞–∑–∞—Ç—å, –º–æ–º–µ–Ω—Ç, –Ω–∞—Å—Ç–æ—è—â–∏–π, —Ä–∞–∑–∂–∏–≥–∞—Ç—å, –ª—é–±–æ–≤—å...      0        0     0   
11  [–ø–æ–µ—Ö–∞—Ç—å, —Å–≤–∞–¥—å–±–∞, –±—ã—Ç—å, –µ–¥–∏–Ω—Å—Ç–≤–µ–Ω–Ω—ã–π, –æ–¥–∏–Ω–æ–∫–∏...      0        0     0   
12                     [—É–º–∏—Ä–∞—Ç—å, –¥–µ–Ω—å, –æ–±–µ–∑–≤–æ–∂–∏–≤–∞–Ω–∏–µ]      0        0     0   
13  [–ø—Ä–æ—Å—Ç–æ, –∏–º—è, —Ä–∏—Ñ–º–æ–≤–∞—Ç—å, —Ñ–æ–Ω–æ–≤—ã–π, —Ä—ç–ø, —á—É–≤–∞–∫, ...      0        0     0   
14  [–ª–æ–ª–∞, –º–µ—á—Ç–∞—Ç—å, –ø—Ä–∏—è—Ç–µ–ª—å, —Å–µ–≥–æ–¥–Ω—è, —Ç—ã, –¥–æ—Å—Ç–∞—Ç–æ...      1        0     0   
15  [–æ—Ä–¥–∏–Ω–∞—Ç–æ—Ä, –∞–Ω–µ—Å—Ç–µ–∑–∏–æ–ª–æ–≥, —ç—Ç–æ, –∑–∞—Å—Ç–∞–≤–ª—è—Ç—å, –≤—ã–¥...      1        0     0   
16  [–æ–±—ä—è–≤–ª—è—Ç—å, –≤–æ–π–Ω–∞, –±–æ–ª—å—à–æ–π, –ø–∞–ª–µ—Ü, –¥–∞–Ω–≥–∏—Ç, –∏–º—è...      0        0     0   
17  [—Å–ª—ã—à–∞—Ç—å, –ø—Ä–∏—á–∏–Ω–∞, –∏–∑–Ω–∞—á–∞–ª—å–Ω–æ, –æ–∑–∞–±–æ—á–µ–Ω–Ω—ã–π, –∑–∞...      0        0     0   
18                           [–∏–º—è, –ª–µ–≥–µ–Ω–¥–∞—Ä–Ω—ã–π, –ø–∞–ø–∞]      0        0     0   
19  [–æ–±—è–∑–∞—Ç–µ–ª—å–Ω–æ, –Ω–µ–Ω–∞–≤–∏–¥–µ—Ç—å, –Ω—Ä–∞–≤–∏—Ç—å—Å—è, —á–µ–ª–æ–≤–µ–∫, ...      1        0     0   
20  [—Ö–æ–∑—è–∏–Ω, –æ–±—ä–µ–¥–∏–Ω—è—Ç—å—Å—è, –∑–≤–µ–∑–¥–∞, –Ω–∞–≥—Ä–∞–¥–∞, –∑–∞–≤—Ç—Ä–∞...      0        0     0   
21  [–ø—Ä–æ–≥–æ–ª–æ—Å–æ–≤–∞—Ç—å, –ø—Ä–æ—Ç–∏–≤, –ø–æ–Ω–∏–º–∞—Ç—å, –≤–∞—à, –æ–ø—ã—Ç, –ø...      0        0     0   
22            [–Ω–∞—Å—Ç–æ—è—â–∏–π, —Ä–µ–±–µ–Ω–æ–∫, —Ö, —Ä–æ–∂–¥–∞—Ç—å—Å—è, –≥–æ–¥]      1        0     0   
23                           [—Å–∫–∞–∑–∞—Ç—å, —Ç—ã, –Ω—Ä–∞–≤–∏—Ç—å—Å—è]      0        0     0   
24                        [–±–ª–æ–∫, –±–ª–æ–∫, –±–ª–æ–∫, –∑–∞–¥–Ω–∏—Ü–∞]      0        0     0   
25  [–æ—Ö, –±–ª–∏–Ω, –∂–∞–ª—å, —ç—Ç–æ, —Å–ª—ã—à–∞—Ç—å, —ç—Ç–æ, —Ç–≤–æ–π, –±–ª–∏–∑...      0        0     0   
26               [–¥–µ–π—Å—Ç–≤–∏—Ç–µ–ª—å–Ω–æ, –≥–ª—É–ø—ã–π, –≤–µ—Ä–∏—Ç—å, —ç—Ç–æ]      1        0     0   
27                       [—á—É–≤–∞–∫, —á—É–≤–∞–∫, —á—É–≤–∞–∫, —á—É–≤–∞–∫]      0        0     0   
28               [—Å–ø–∞—Å–∏–±–æ, –æ—á–µ–Ω—å, —Ü–µ–Ω–∏—Ç—å, –≤–∞—à, –æ—Ç–≤–µ—Ç]      0        0     0   
29                            [—Ö–∏–º–∏—è, –∫–∞–≤—ã–π, —Å–∏–∞–∫–∞–º–∞]      0        0     0   

    joy  sadness  surprise  neutral hashtags mentions  ...  num_exclamation  \
0     0        0         0        1       []       []  ...                0   
1     1        0         0        1       []       []  ...                1   
2     0        1         0        0       []       []  ...                1   
3     1        0         0        1       []       []  ...                0   
4     1        0         0        0       []       []  ...                0   
5     1        0         0        0       []       []  ...                0   
6     0        0         0        0       []       []  ...                0   
7     0        0         0        1       []       []  ...                1

In [ ]:
d = defaultdict(int)

texts = list(df.text)
for text in texts:
  for word in text:
    d[word] += 1



In [ ]:
d = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))

In [ ]:
d = dict(sorted(d.items(), key=lambda item: item[1], reverse=False))

In [ ]:
d

{'–∑–∞–ø—Ä–æ–≥—Ä–∞–º–º–∏—Ä–æ–≤–∞—Ç—å': 1,
 '–ø–µ—Ä–µ—Ö–≤–∞—Ç—ã–≤–∞—Ç—å': 1,
 '—Å—É–±—Å–∏–¥–∏—Ä–æ–≤–∞—Ç—å—Å—è': 1,
 '—Ä–∞—Å–ø–µ—á–∞—Ç—ã–≤–∞—Ç—å': 1,
 '–æ—Ç–ø—Ä—ã—Å–∫': 1,
 '—Ç—Ä—ç—à': 1,
 '—Ä–∞–∑–∂–∏–≥–∞—Ç—å': 1,
 '–æ–±–µ–∑–≤–æ–∂–∏–≤–∞–Ω–∏–µ': 1,
 '—Ä–∏—Ñ–º–æ–≤–∞—Ç—å': 1,
 '–º–µ—á—Ç–∞—Ç—å': 1,
 '–±—Ä–æ–¥—è—á–∏–π': 1,
 '–æ—Ä–¥–∏–Ω–∞—Ç–æ—Ä': 1,
 '–∞–Ω–µ—Å—Ç–µ–∑–∏–æ–ª–æ–≥': 1,
 '–≤—ã–¥—É–≤–∞—Ç—å': 1,
 '—É—Å–∫–æ—Ä–µ–Ω–∏–µ': 1,
 '–¥–∞–Ω–≥–∏—Ç': 1,
 '–æ–∑–∞–±–æ—á–µ–Ω–Ω—ã–π': 1,
 '—Å—É—Ä–æ–≤–∞': 1,
 '–∫—Ä–µ–¥–∏—Ç–Ω—ã–π': 1,
 '–∫–∞–≤—ã–π': 1,
 '—Å–∏–∞–∫–∞–º–∞': 1,
 '–æ–≤–µ—Ä—Ç–∞–π–º': 1,
 '—á–µ–º–ø–∏–æ–Ω–∞—Ç': 1,
 '—Ñ—É—Ç–±–æ–ª—å–Ω—ã–π': 1,
 '–º–∞–ª—å–∫': 1,
 '–∏–Ω—Ñ–æ—Ä–º–∏—Ä–æ–≤–∞—Ç—å': 1,
 '–ø–µ—Ä–µ–ø–∏—Å—ã–≤–∞—Ç—å—Å—è': 1,
 '—Å–æ–∑—Ä–µ–≤–∞—Ç—å': 1,
 '–∏—Å–ø–µ—á—å': 1,
 '–æ–¥–Ω–æ–¥–Ω–µ–≤–Ω—ã–π': 1,
 '–∞–ª—å–º–æ–Ω—Ç': 1,
 '—Ö—Ä–∞–Ω–∏—Ç—å': 1,
 '–ª–∏—Ç—Ä': 1,
 '—Ñ–µ–Ω—Ç–∞–Ω–∏—Ç—å': 1,
 '—Ä—É–±–µ—Ü': 1,
 '–≤–¥–∞–ª–∏': 1,
 '—Å–±–∞–ª–∞–Ω—Å–∏—Ä–æ–≤–∞—Ç—å': 1,
 '—Å–º—É—â–µ–Ω–Ω—ã–π': 1,
 '–Ω–µ–ø—Ä–µ

In [ ]:
del_list = [k for k, v in d.items() if v == 1]

In [ ]:
df.text

0       [—ç—Ç–æ, –Ω–æ–≤—ã–π, –æ—Ä–ª–µ–∞–Ω, –¥–µ–π—Å—Ç–≤–∏—Ç–µ–ª—å–Ω–æ, —á—É–≤—Å—Ç–≤–æ–≤–∞—Ç...
1       [–∑–Ω–∞—Ç—å, –æ—Ç–≤–µ—Ç, —á—É–≤–∞–∫, –∑–∞–ø—Ä–æ–≥—Ä–∞–º–º–∏—Ä–æ–≤–∞—Ç—å, –ø–µ—Ä–µ—Ö...
2                                        [–∂–∏–∑–Ω—å, –≥—Ä—É—Å—Ç–Ω–æ]
3       [—ç–∫–æ–Ω–æ–º–∏–∫–∞, –Ω–∞—Ö–æ–¥–∏—Ç—å—Å—è, –∂–µ—Å—Ç–∫–∏–π, –∫–æ–Ω—Ç—Ä–æ–ª—å, —Å—É–±...
4       [–º–æ—á—å, –ª–µ–≥–∫–æ, –≤–∑—è—Ç—å, –Ω–∞—Å—Ç–æ—è—â–∏–π, –∫–∞–º–µ—Ä–∞, –∑–∞–∫–æ–Ω–Ω...
                              ...                        
5421    [—ç—Ç–æ, –¥–æ–≤–æ–ª—å–Ω–æ, –æ–ø–∞—Å–Ω—ã–π, –≥–æ—Å—É–¥–∞—Ä—Å—Ç–≤–æ, —Ä–µ—à–∞—Ç—å, ...
5422    [—Å–µ–≥–æ–¥–Ω—è, —É—Ç—Ä–æ, –ø–æ–¥–∞–≤–∞—Ç—å, —Ä–∞–∑–≤–æ–¥, –Ω–∞–¥–µ—è—Ç—å—Å—è, —É...
5423    [–ø–æ—Å–ª–µ–¥–Ω–∏–π, —ç—Ç–æ, —Å–ª—É—á–∞—Ç—å—Å—è, –ø—Ä–æ—Å—Ç–æ, —Å–∫–∞–∑–∞—Ç—å, –∑...
5424    [—Ç–µ—Ä–ø–µ—Ç—å, –º–æ—á—å, –≤—ã—Å–æ–∫–æ–º–µ—Ä–Ω—ã–π, –ø—Ä–∏–¥—É—Ä–æ–∫, —Ñ–æ—Ä–¥, ...
5425                          [–Ω—Ä–∞–≤–∏—Ç—å—Å—è, –¥–µ—Ç—Å–∫–∏–π, —á–µ–ª–∫–∞]
Name: text, Length: 5417, dtype: object

In [ ]:
def del_rare_words(text_lem, del_list=del_list):
  return [word for word in text_lem if word not in del_list]

In [ ]:
df.text = df.text.map(del_rare_words)

In [ ]:
df['len_lem'] = len(df.text)

In [ ]:
# df.to_csv('valid_lemma_df_1.csv')

# from google.colab import files
# files.download('valid_lemma_df_1.csv')

In [ ]:
d = defaultdict(int)

texts = list(df.text)
for text in texts:
  for word in text:
    d[word] += 1


In [ ]:
d = dict(sorted(d.items(), key=lambda item: item[1], reverse=False))
d

{'–æ—Ä–ª–µ–∞–Ω': 2,
 '–∂–µ—Å—Ç–∫–∏–π': 2,
 '–Ω—é–∞–Ω—Å': 2,
 '—Å–µ–≥–æ–¥–Ω—è—à–Ω–∏–π': 2,
 '—Å—ã–Ω–æ–∫': 2,
 '–∑–∞–≤–æ–µ–≤—ã–≤–∞—Ç—å': 2,
 '—Ñ–æ–Ω–æ–≤—ã–π': 2,
 '–∑–∞—Å—Ç—Ä–µ–≤–∞—Ç—å': 2,
 '–∫—É—Ä–µ–Ω–∏–µ': 2,
 '–Ω–æ—Å': 2,
 '–∏–∑–Ω–∞—á–∞–ª—å–Ω–æ': 2,
 '—Ä–∞–∑–º–Ω–æ–∂–∞—Ç—å—Å—è': 2,
 '–Ω–∞–≥—Ä–∞–¥–∞': 2,
 '—Ä—É–ª–µ—Ç–∫–∞': 2,
 '–±–ª–∏–∑–∫–∏–µ': 2,
 '—Ö–∏–º–∏—è': 2,
 '–≤–æ—Å–∫—Ä–µ—Å–Ω—ã–π': 2,
 '–Ω–∞—Ö–æ–¥–∫–∞': 2,
 '—Ç–æ—Ä–≥–æ–≤—ã–π': 2,
 '–æ—à–∏–±–æ—á–Ω—ã–π': 2,
 '–Ω–µ–ø–æ–ø—É–ª—è—Ä–Ω—ã–π': 2,
 '–≤—ã–ø–µ—á–∫–∞': 2,
 '–Ω–µ–≤–æ–∑–º—É—Ç–∏–º—ã–π': 2,
 '–∞–ø—Ç–µ–∫–∞': 2,
 '–ø–æ–≤—Ç–æ—Ä–µ–Ω–∏–µ': 2,
 '—à–∏—Ä–æ–∫–∏–π': 2,
 '–ø–∞–Ω': 2,
 '–¥–µ—Ç–∫–∞': 2,
 '–º—Ä–∞—á–Ω—ã–π': 2,
 '—Å–≤–µ—Ç–ª—ã–π': 2,
 '–Ω–æ–≤–µ–Ω—å–∫–∏–π': 2,
 '–∫—Ä—ã—Å–∞': 2,
 '–¥–∂': 2,
 '—Ö—Ä—É–ø–∫–∏–π': 2,
 '–º–∞—Ñ–∏—è': 2,
 '–≤–æ—Å—Ö–∏—â–∞—Ç—å—Å—è': 2,
 '–ø—Ä–æ–≤–µ—Ä—è—Ç—å—Å—è': 2,
 '—É–≤–∞–∂–∞—Ç—å': 2,
 '–Ω–µ–±–ª–æ–∫–∏—Ä–æ–≤–∞—Ç—å': 2,
 '–∑–Ω–∞–∫–æ–º—Å—Ç–≤–æ': 2,
 '–ø–∞–¥–∞—Ç—å': 2,
 '–≥–µ–Ω–æ—Ü–∏–¥': 2,
 '–∂–∏—Ç–µ–ª—å': 2

In [ ]:
df.text = df.text.str.join(' ')

In [ ]:
df.to_csv('prep_valid.csv')

In [ ]:
from google.colab import files
files.download('prep_valid.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>